In [ ]:
# table
# prompt for a url and web (clean)
# prompt for pages you want to retrieved
### from_id



# href (absoulte reference) 
## parse and insert url and html
## retrieve all tags and clean 
###toid
# insert into links

NoneType

In [60]:
import sqlite3
import urllib.error
import ssl
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Pages
    (id INTEGER PRIMARY KEY, url TEXT UNIQUE, html TEXT,
     error INTEGER, old_rank REAL, new_rank REAL)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Links
    (from_id INTEGER, to_id INTEGER)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Webs (url TEXT UNIQUE)''')

# Check to see if we are already in progress...
cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
row = cur.fetchone()
if row is not None:
    print("Restarting existing crawl.  Remove spider.sqlite to start a fresh crawl.")
else :
    starturl = input('Enter web url or enter: ')
    if ( len(starturl) < 1 ) : starturl = 'http://www.dr-chuck.com/'
    if ( starturl.endswith('/') ) : starturl = starturl[:-1]
    web = starturl
    if ( starturl.endswith('.htm') or starturl.endswith('.html') ) :
        pos = starturl.rfind('/')
        web = starturl[:pos]

    if ( len(web) > 1 ) :
        cur.execute('INSERT OR IGNORE INTO Webs (url) VALUES ( ? )', ( web, ) )
        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( starturl, ) )
        conn.commit()

# Get the current webs
cur.execute('''SELECT url FROM Webs''')
webs = list()
for row in cur:
    webs.append(str(row[0]))

print(webs)

many = 0
while True:
    if ( many < 1 ) :
        sval = input('How many pages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)
    many = many - 1

    cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
    try:
        row = cur.fetchone()
        # print row
        fromid = row[0]
        url = row[1]
    except:
        print('No unretrieved HTML pages found')
        many = 0
        break

    print(fromid, url, end=' ')

    # If we are retrieving this page, there should be no links from it
    cur.execute('DELETE from Links WHERE from_id=?', (fromid, ) )
    try:
        document = urlopen(url, context=ctx)

        html = document.read()
        if document.getcode() != 200 :
            print("Error on page: ",document.getcode())
            cur.execute('UPDATE Pages SET error=? WHERE url=?', (document.getcode(), url) )

        if 'text/html' != document.info().get_content_type() :
            print("Ignore non text/html page")
            cur.execute('DELETE FROM Pages WHERE url=?', ( url, ) )
            cur.execute('UPDATE Pages SET error=0 WHERE url=?', (url, ) )
            conn.commit()
            continue

        print('('+str(len(html))+')', end=' ')

        soup = BeautifulSoup(html, "html.parser")
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except:
        print("Unable to retrieve or parse page")
        cur.execute('UPDATE Pages SET error=-1 WHERE url=?', (url, ) )
        conn.commit()
        continue

    cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( url, ) )
    cur.execute('UPDATE Pages SET html=? WHERE url=?', (memoryview(html), url ) )
    conn.commit()

    # Retrieve all of the anchor tags
    tags = soup('a')
    count = 0
    for tag in tags:
        href = tag.get('href', None)
        if ( href is None ) : continue
        # Resolve relative references like href="/contact"
        up = urlparse(href)
        if ( len(up.scheme) < 1 ) :
            href = urljoin(url, href)
        ipos = href.find('#')
        if ( ipos > 1 ) : href = href[:ipos]
        if ( href.endswith('.png') or href.endswith('.jpg') or href.endswith('.gif') ) : continue
        if ( href.endswith('/') ) : href = href[:-1]
        # print href
        if ( len(href) < 1 ) : continue

		# Check if the URL is in any of the webs
        found = False
        for web in webs:
            if ( href.startswith(web) ) :
                found = True
                break
        if not found : continue

        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( href, ) )
        count = count + 1
        conn.commit()

        cur.execute('SELECT id FROM Pages WHERE url=? LIMIT 1', ( href, ))
        try:
            row = cur.fetchone()
            toid = row[0]
        except:
            print('Could not retrieve id')
            continue
        # print fromid, toid
        cur.execute('INSERT OR IGNORE INTO Links (from_id, to_id) VALUES ( ?, ? )', ( fromid, toid ) )


    print(count)

cur.close()


Enter web url or enter: 
['100', 'http://www.dr-chuck.com']
How many pages:1000
No unretrieved HTML pages found


In [56]:
# intuition
# form lists called links, from_id and to_id gathering from_id and to_id from dbs
# notice that we only interest in the page which has from_id as
# well as to_id
# generate dict prev_ranks to gather from id
# generate dict gives_ranks to gather to id
# count the amount of old rank gives to the next rank, as well as the total
# add the amount to the next rank
#  computing the evporation and redistribute the evaporation
# compute the convergence
# update the database

import sqlite3

conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

from_ids = list()
row = cur.execute('''SELECT DISTINCT from_id FROM Links''')
for row in cur:
    from_ids.append(row[0])

to_ids = list()
links = list()
cur.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
for row in cur:
    from_id = row[0]
    to_id = row[1]
    if from_id == to_id : continue
    if from_id not in from_ids : continue
    if to_id not in from_ids : continue
    links.append(row)
    if to_id not in to_ids : to_ids.append(to_id)

prev_ranks = dict()
for node in from_ids:
    cur.execute('''SELECT new_rank FROM Pages WHERE id = ?
    ''', (node, ))
    row = cur.fetchone()
    prev_ranks[node] = row[0]
    
many = 1
sval = input('How many iterations?')
if len(sval) > 0:
    many = int(sval)
if len(prev_ranks) < 1:
    print("Nothing to page rank. Check data.")
    quit()

for i in range(many):
    total = 0.0
    next_ranks = dict()
    for (node, old_rank) in list(prev_ranks.items()):
        total = total + old_rank
        next_ranks[node] = 0.0
    for (node, old_rank) in list(prev_ranks.items()):
        give_ids = list()
        for (from_id, to_id) in links:
            if from_id != node: continue
            if to_id not in to_ids: continue
            give_ids.append(to_id)
        if len(give_ids) < 1 : continue
        amount = total/len(give_ids)
        
        for id in give_ids:
            next_ranks[id] = amount + next_ranks[id]
    newtot = 0
    for (node, new_rank) in list(next_ranks.items()):
        newtot = newtot + new_rank
    evap = total - newtot
    for (node, new_rank) in list(next_ranks.items()):
        next_ranks[node] = next_ranks[node] + evap
    
    new_tot = 0
    for (node, new_rank) in list(next_ranks.items()):
        newtot = newtot + new_rank
    
    todiff = 0
    for (node, old_rank) in list(prev_ranks.items()):
        old_rank = prev_ranks[node] 
        new_rank = next_ranks[node]
        diff = abs(old_rank - new_rank)
        todiff = diff + todiff
    
    avediff = todiff/len(prev_ranks)
    print(i+1, avediff)
    
    prev_ranks = next_ranks
    
print(list(next_ranks.items())[:50])
cur.execute('''UPDATE Pages SET old_rank = new_rank''')
for (id, next_rank) in list(next_ranks.items()):
    cur.execute('''UPDATE Pages Set new_rank = ? WHERE id = ?''', (new_rank, id))
conn.commit()
cur.close()
        
        
        
        
        

How many iterations?100


TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [62]:
import sqlite3
import urllib.error
import ssl
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Pages
    (id INTEGER PRIMARY KEY, url TEXT UNIQUE, html TEXT,
     error INTEGER, old_rank REAL, new_rank REAL)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Links
    (from_id INTEGER, to_id INTEGER)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Webs (url TEXT UNIQUE)''')

# Check to see if we are already in progress...
cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
row = cur.fetchone()
if row is not None:
    print("Restarting existing crawl.  Remove spider.sqlite to start a fresh crawl.")
else :
    starturl = input('Enter web url or enter: ')
    if ( len(starturl) < 1 ) : starturl = 'http://www.dr-chuck.com/'
    if ( starturl.endswith('/') ) : starturl = starturl[:-1]
    web = starturl
    if ( starturl.endswith('.htm') or starturl.endswith('.html') ) :
        pos = starturl.rfind('/')
        web = starturl[:pos]

    if ( len(web) > 1 ) :
        cur.execute('INSERT OR IGNORE INTO Webs (url) VALUES ( ? )', ( web, ) )
        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( starturl, ) )
        conn.commit()

# Get the current webs
cur.execute('''SELECT url FROM Webs''')
webs = list()
for row in cur:
    webs.append(str(row[0]))

print(webs)

many = 0
while True:
    if ( many < 1 ) :
        sval = input('How many pages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)
    many = many - 1

    cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
    try:
        row = cur.fetchone()
        # print row
        fromid = row[0]
        url = row[1]
    except:
        print('No unretrieved HTML pages found')
        many = 0
        break

    print(fromid, url, end=' ')

    # If we are retrieving this page, there should be no links from it
    cur.execute('DELETE from Links WHERE from_id=?', (fromid, ) )
    try:
        document = urlopen(url, context=ctx)

        html = document.read()
        if document.getcode() != 200 :
            print("Error on page: ",document.getcode())
            cur.execute('UPDATE Pages SET error=? WHERE url=?', (document.getcode(), url) )

        if 'text/html' != document.info().get_content_type() :
            print("Ignore non text/html page")
            cur.execute('DELETE FROM Pages WHERE url=?', ( url, ) )
            cur.execute('UPDATE Pages SET error=0 WHERE url=?', (url, ) )
            conn.commit()
            continue

        print('('+str(len(html))+')', end=' ')

        soup = BeautifulSoup(html, "html.parser")
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except:
        print("Unable to retrieve or parse page")
        cur.execute('UPDATE Pages SET error=-1 WHERE url=?', (url, ) )
        conn.commit()
        continue

    cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( url, ) )
    cur.execute('UPDATE Pages SET html=? WHERE url=?', (memoryview(html), url ) )
    conn.commit()

    # Retrieve all of the anchor tags
    tags = soup('a')
    count = 0
    for tag in tags:
        href = tag.get('href', None)
        if ( href is None ) : continue
        # Resolve relative references like href="/contact"
        up = urlparse(href)
        if ( len(up.scheme) < 1 ) :
            href = urljoin(url, href)
        ipos = href.find('#')
        if ( ipos > 1 ) : href = href[:ipos]
        if ( href.endswith('.png') or href.endswith('.jpg') or href.endswith('.gif') ) : continue
        if ( href.endswith('/') ) : href = href[:-1]
        # print href
        if ( len(href) < 1 ) : continue

		# Check if the URL is in any of the webs
        found = False
        for web in webs:
            if ( href.startswith(web) ) :
                found = True
                break
        if not found : continue

        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( href, ) )
        count = count + 1
        conn.commit()

        cur.execute('SELECT id FROM Pages WHERE url=? LIMIT 1', ( href, ))
        try:
            row = cur.fetchone()
            toid = row[0]
        except:
            print('Could not retrieve id')
            continue
        # print fromid, toid
        cur.execute('INSERT OR IGNORE INTO Links (from_id, to_id) VALUES ( ?, ? )', ( fromid, toid ) )


    print(count)

cur.close()


Restarting existing crawl.  Remove spider.sqlite to start a fresh crawl.
['http://www.dr-chuck.com']


KeyboardInterrupt: 

In [66]:
import sqlite3

conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

# Find the ids that send out page rank - we only are interested
# in pages in the SCC that have in and out links
cur.execute('''SELECT DISTINCT from_id FROM Links''')
from_ids = list()
for row in cur: 
    from_ids.append(row[0])

# Find the ids that receive page rank 
to_ids = list()
links = list()
cur.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
for row in cur:
    from_id = row[0]
    to_id = row[1]
    if from_id == to_id : continue
    if from_id not in from_ids : continue
    if to_id not in from_ids : continue
    links.append(row)
    if to_id not in to_ids : to_ids.append(to_id)

# Get latest page ranks for strongly connected component
prev_ranks = dict()
for node in from_ids:
    cur.execute('''SELECT new_rank FROM Pages WHERE id = ?''', (node, ))
    row = cur.fetchone()
    prev_ranks[node] = row[0]

sval = input('How many iterations:')
many = 1
if ( len(sval) > 0 ) : many = int(sval)

# Sanity check
if len(prev_ranks) < 1 : 
    print("Nothing to page rank.  Check data.")
    quit()

# Lets do Page Rank in memory so it is really fast
for i in range(many):
    # print prev_ranks.items()[:5]
    next_ranks = dict();
    total = 0.0
    for (node, old_rank) in list(prev_ranks.items()):
        total = total + old_rank
        next_ranks[node] = 0.0
    # print total

    # Find the number of outbound links and sent the page rank down each
    for (node, old_rank) in list(prev_ranks.items()):
        # print node, old_rank
        give_ids = list()
        for (from_id, to_id) in links:
            if from_id != node : continue
           #  print '   ',from_id,to_id

            if to_id not in to_ids: continue
            give_ids.append(to_id)
        if ( len(give_ids) < 1 ) : continue
        amount = old_rank / len(give_ids)
        # print node, old_rank,amount, give_ids
    
        for id in give_ids:
            next_ranks[id] = next_ranks[id] + amount
    
    newtot = 0
    for (node, next_rank) in list(next_ranks.items()):
        newtot = newtot + next_rank
    evap = (total - newtot) / len(next_ranks)

    # print newtot, evap
    for node in next_ranks:
        next_ranks[node] = next_ranks[node] + evap

    newtot = 0
    for (node, next_rank) in list(next_ranks.items()):
        newtot = newtot + next_rank

    # Compute the per-page average change from old rank to new rank
    # As indication of convergence of the algorithm
    totdiff = 0
    for (node, old_rank) in list(prev_ranks.items()):
        new_rank = next_ranks[node]
        diff = abs(old_rank-new_rank)
        totdiff = totdiff + diff

    avediff = totdiff / len(prev_ranks)
    print(i+1, avediff)

    # rotate
    prev_ranks = next_ranks

# Put the final ranks back into the database
print(list(next_ranks.items())[:5])
cur.execute('''UPDATE Pages SET old_rank=new_rank''')
for (id, new_rank) in list(next_ranks.items()) :
    cur.execute('''UPDATE Pages SET new_rank=? WHERE id=?''', (new_rank, id))
conn.commit()
cur.close()



How many iterations:1000
1 7.771561172376095e-17
2 7.771561172376095e-17
3 7.771561172376095e-17
4 7.771561172376095e-17
5 7.771561172376095e-17
6 7.771561172376095e-17
7 7.771561172376095e-17
8 7.771561172376095e-17
9 7.771561172376095e-17
10 7.771561172376095e-17
11 7.771561172376095e-17
12 7.771561172376095e-17
13 7.771561172376095e-17
14 7.771561172376095e-17
15 7.771561172376095e-17
16 7.771561172376095e-17
17 7.771561172376095e-17
18 7.771561172376095e-17
19 7.771561172376095e-17
20 7.771561172376095e-17
21 7.771561172376095e-17
22 7.771561172376095e-17
23 7.771561172376095e-17
24 7.771561172376095e-17
25 7.771561172376095e-17
26 7.771561172376095e-17
27 7.771561172376095e-17
28 7.771561172376095e-17
29 7.771561172376095e-17
30 7.771561172376095e-17
31 7.771561172376095e-17
32 7.771561172376095e-17
33 7.771561172376095e-17
34 7.771561172376095e-17
35 7.771561172376095e-17
36 7.771561172376095e-17
37 7.771561172376095e-17
38 7.771561172376095e-17
39 7.771561172376095e-17
40 7.7715

931 7.771561172376095e-17
932 7.771561172376095e-17
933 7.771561172376095e-17
934 7.771561172376095e-17
935 7.771561172376095e-17
936 7.771561172376095e-17
937 7.771561172376095e-17
938 7.771561172376095e-17
939 7.771561172376095e-17
940 7.771561172376095e-17
941 7.771561172376095e-17
942 7.771561172376095e-17
943 7.771561172376095e-17
944 7.771561172376095e-17
945 7.771561172376095e-17
946 7.771561172376095e-17
947 7.771561172376095e-17
948 7.771561172376095e-17
949 7.771561172376095e-17
950 7.771561172376095e-17
951 7.771561172376095e-17
952 7.771561172376095e-17
953 7.771561172376095e-17
954 7.771561172376095e-17
955 7.771561172376095e-17
956 7.771561172376095e-17
957 7.771561172376095e-17
958 7.771561172376095e-17
959 7.771561172376095e-17
960 7.771561172376095e-17
961 7.771561172376095e-17
962 7.771561172376095e-17
963 7.771561172376095e-17
964 7.771561172376095e-17
965 7.771561172376095e-17
966 7.771561172376095e-17
967 7.771561172376095e-17
968 7.771561172376095e-17
969 7.771561

In [ ]:
# independently do 
from urllib.parse import urlparse
from urllib.parse import urljoin
from urllib.request import urlopen
import urllib.error
from bs4 import BeautifulSoup 
import ssl
import sqlite3
import json

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('jayspider.sqlite')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Pages (id INTEGER PRIMARY KEY, url TEXT UNIQUE, html TEXT, 
            error INTEGER, new_rank REAL, old_rank REAL)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Links (from_id INTEGER, to_id INTEGER )''')
cur.execute('''CREATE TABLE IF NOT EXISTS Webs (url TEXT UNIQUE)''')

cur.execute('''SELECT id, url FROM Pages WHERE html is Null AND url is Null ORDER BY RANDOM() LIMIT 1''')
row = cur.fetchone()
if row != None:
    print('dj')
else:
    starturl = input('Enter a URL')
    if len(starturl) < 1: starturl = 'http://www.dr-chuck.com/'
    if starturl.endswith('/'): starturl = starturl[:-1]
    web = starturl
    if starturl.endswith('.htm') or starturl.endswith('.html'):
        pos = starturl.rfind('/')
        web = starurl[:pos]
    if len(web) > 1:
        cur.execute('''INSERT OR IGNORE INTO Webs (url) VALUES (?)''', (web, ))
        cur.execute('''INSERT OR IGNORE INTO Pages(url, html, new_rank) VALUES(?, Null, 1.0)''', (starturl, ))
        conn.commit()
cur.execute('''SELECT url FROM Webs''')    
webs = list()
for row in cur:
    webs.append(row[0])
print(webs)    
many = 0
while True:
    if many < 1:
        sval = input('How many Pages: ')
        if len(sval) < 1: break
        many = int(sval)
    many = many - 1
    
    cur.execute('''SELECT id,url FROM Pages WHERE html is NULL and error is Null ORDER BY RANDOM() LIMIT 1''') 
    try:
        row = cur.fetchone() 
        from_id = row[0]
        url = row[1]
    except:
        many = 0
        print('Unable to')
        break
  
    
    cur.execute('''DELETE FROM Links WHERE from_id = ?''', (from_id, )) 
    try:
        document = urlopen(url, context = ctx)
        html = document.read()
        if document.getcode() != 200:
            cur.execute('''UPDATE Pages Set error = ? WHERE url = ?''', (document.getcode(), url)) 
        if 'text/html' != document.info().get_content_type():
            cur.execute('''DELETE FROM Pages WHERE url=?''', (url, )) 
            cur.execute('''UPDATE Pages set error = 0 WHERE url = ?''', (url, )) 
            conn.commit()
            continue
        print('('+str(len(html))+')', end=' ')
        soup = BeautifulSoup(html, 'html.parser')
        
    except KeyboardInterrupt:
        
        break
    except: 
        cur.execute('''UPDATE Pages Set error = -1 WHERE url = ?''', (url, )) 
        conn.commit()
        continue
        
    cur.execute('''INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES(?, Null, 1.0)''', (url, )) 
    cur.execute('''UPDATE Pages Set html=? WHERE url = ?''', (memoryview(html), url )) 
    conn.commit()
    
    count = 0
    tags = soup('a')
    for tag in tags:
        href = tag.get('href', None)
        if href is None: continue
        up = urlparse(href)
        if len(up.scheme) < 1: 
            href = urljoin(url,href)
        ipos = href.find('#')
        if (ipos) > 1:
            href = href[:ipos]
        if href.endswith('.jpg') or href.endswith('.png') or href.endswith('.gif'):
            continue
        if href.endswith('/'): 
            href = href[:-1]
        if len(href) < 1:
            continue

        found = False
        for web in webs:
            if ( href.startswith(web) ) :
                found = True
                break
        if not found : continue        
        cur.execute('''INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES(?, Null, 1.0)''', (href, )) 
        count = count + 1
        conn.commit()
        
        cur.execute('''SELECT id FROM Pages WHERE  url = ? LIMIT 1''', (href, )) 
        try:
            row = cur.fetchone()
            to_id = row[0]
        except:
            print('Could not retrieve id')
            continue
        cur.execute('''INSERT OR IGNORE INTO Links (from_id, to_id) VALUES(?, ?)''', (from_id, to_id)) 
        conn.commit()
    print(count)
        
cur.close()    
    

Enter a URL
['http://www.dr-chuck.com']
How many Pages: 4
(8769) 4
(5843) 4
(2426) 1


In [40]:
found = False
r = ['webs', 'jsfs', 'jsfla', 'sjfl']
for web in r:
    if ( r.startswith('we') ) :
        found = True
        break
if not found : 
    print(found)
    continue


AttributeError: 'list' object has no attribute 'startswith'